## Задание

Вам необходимо построить модель, которая на основании данных, поступающих каждую
минуту, определяют качество продукции, производимое на обжиговой машине.
Обжиговая машина представляет собой агрегат, состоящий из 5 одинаковых по размеру
камер, в каждой камере установлено по 3 датчика температур. Кроме этого, для данной
задачи Вы собрали данные о высоте слоя сырья и его влажности. Высота слоя и влажность
измеряются при входе сырья в машину. Сырье проходит через обжиговую машину за час.
Данные с показателями работы обжиговой машины содержатся в файле X_data.csv:

T_data_1_1 : 1-й датчик в 1-й камере
<br>T_data_1_2 : 2-й датчик в 1-й камере
<br>T_data_1_3 : 3-й датчик в 1-й камере
<br>T_data_2_1 : 1-й датчик во 2-й камере
<br>T_data_2_2 : 2-й датчик во 2-й камере
<br>T_data_2_3 : 3-й датчик во 2-й камере
<br>T_data_3_1 : 1-й датчик в 3-й камере
<br>T_data_3_2 : 2-й датчик в 3-й камере
<br>T_data_3_3 : 3-й датчик в 3-й камере
<br>T_data_4_1 : 1-й датчик в 4-й камере
<br>T_data_4_2 : 2-й датчик в 4-й камере
<br>T_data_4_3 : 3-й датчик в 4-й камере
<br>T_data_5_1 : 1-й датчик в 5-й камере
<br>T_data_5_2 : 2-й датчик в 5-й камере
<br>T_data_5_3 : 3-й датчик в 5-й камере
<br>H_data : Высота слоя
<br>AH_data : Влажность сырья

Качество продукции измеряется в лаборатории по пробам, которые забираются каждый
час, данные по известным анализам содержатся в файле Y_train.csv. В файле указано
время забора пробы, проба забирается на выходе из обжиговой машины.
Вы договорились с заказчиком, что оценкой модели будет являться показатель MAE, для
оценки модели необходимо сгенерировать предсказания за период, указанный в файле
Y_submit.csv (5808 предиктов).

Versions:
<br>python 3.7.1
<br>numpy 1.16.2
<br>pandas 0.23.4
<br>sklearn 0.20.1
<br>lightgbm 2.2.3
<br>catboost 0.13.1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import re

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

import warnings
warnings.simplefilter('ignore')

In [4]:
%%time
raw_data = pd.read_csv('X_data.csv', sep=';', index_col=0)
raw_target = pd.read_csv('Y_train.csv', sep=';', index_col=0, header=None)
raw_test = pd.read_csv('Y_submit.csv', sep=';', index_col=0, header=None)

Wall time: 16.1 s


In [5]:
raw_data.head(3)

,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
2015-01-01 00:00:00,212,210,211,347,353,347,474,473,481,346,348,355,241,241,243,167.85,9.22
2015-01-01 00:01:00,212,211,211,346,352,346,475,473,481,349,348,355,241,241,243,162.51,9.22
2015-01-01 00:02:00,212,211,211,345,352,346,476,473,481,352,349,355,242,241,242,164.99,9.22


In [6]:
raw_target.columns = ['target']
raw_target.index.name = 'timestamp'
raw_target.head(3)

,target
timestamp,
2015-01-04 00:05:00,392
2015-01-04 01:05:00,384
2015-01-04 02:05:00,393


In [7]:
raw_test.columns = ['target']
raw_test.index.name = 'timestamp'
raw_test.head(3)

,target
timestamp,
2018-05-04 00:05:00,420
2018-05-04 01:05:00,420
2018-05-04 02:05:00,420


In [8]:
raw_data.shape, raw_target.shape, raw_test.shape

((2103841, 17), (29184, 1), (5808, 1))

Значения целевой переменной даны на моменты времени H:05:00, где H $\in [0:23]$. В качестве признаков будем использовать средние и максимальные значения температур датчиков за временные интервалы [H:06:00; (H+1):05:00].

In [9]:
%%time

# удалим первые 6 строк, чтобы данные начинались с 6-й минуты
train = raw_data.drop(index=raw_data.index[:6]).reset_index()

# будем агрегировать по столбцу timestamp
train.rename({'index': 'timestamp'}, axis=1, inplace=True)
train['timestamp'] = pd.to_datetime(train['timestamp'])

# для удобства агрегации сдвинем время на 6 минут назад, чтобы начало было в HH:00:00
def time_shift(time, hours, minutes):
    """
    Возвращает дату, сдвинутую на 'hours' часов и 'minutes' минут назад
    """
    return dt.datetime.strptime(str(time), "%Y-%m-%d %H:%M:%S") - dt.timedelta(hours=hours, minutes=minutes)

train['timestamp'] = train['timestamp'].apply(time_shift, args=(0, 6))

#удалим слово 'data' из названия колонок
train.columns = [re.sub(r"_data", "", col) for col in train.columns]

Wall time: 45.4 s


In [10]:
train.head(3)

,timestamp,T_1_1,T_1_2,T_1_3,T_2_1,T_2_2,T_2_3,T_3_1,T_3_2,T_3_3,T_4_1,T_4_2,T_4_3,T_5_1,T_5_2,T_5_3,H,AH
0,2015-01-01 00:00:00,213,212,211,341,349,346,480,473,482,363,350,354,244,241,242,166.14,9.22
1,2015-01-01 00:01:00,213,212,211,340,348,345,482,473,482,365,350,354,244,241,242,164.38,9.22
2,2015-01-01 00:02:00,213,212,211,339,347,345,483,473,482,367,350,354,244,241,242,163.89,9.22


In [11]:
# Т.к. высота и влажность сырья замеряются только на входе в печь, агрегировать по часу будем только показания 
#датчиков температуры. Высоту и влажность сырья отделим временно в отдельный датафрейм
start_features = ['H', 'AH']
start_feat_data = train[start_features]
start_feat_data.index = train['timestamp']

temp_features = [feat for feat in train.columns if feat not in start_features]
temp_data_gr = train[temp_features].groupby(pd.Grouper(key='timestamp', freq='H')).agg(['mean', 'max'])
temp_data_gr.columns = temp_data_gr.columns.map('_'.join).str.strip('_')

# соединяем агрегированные температурные признаки с замерами высоты и влажности 
train_gr = temp_data_gr.merge(start_feat_data, left_index=True, right_index=True)
train_gr.head(3)

,T_1_1_mean,T_1_1_max,T_1_2_mean,T_1_2_max,T_1_3_mean,T_1_3_max,T_2_1_mean,T_2_1_max,T_2_2_mean,T_2_2_max,...,T_4_3_mean,T_4_3_max,T_5_1_mean,T_5_1_max,T_5_2_mean,T_5_2_max,T_5_3_mean,T_5_3_max,H,AH
timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,212.916667,214,213.716667,215,211.183333,212,316.200000,341,324.000000,349,...,349.116667,354,246.75,249,240.866667,241,239.366667,242,166.14,9.22
2015-01-01 01:00:00,210.133333,215,202.600000,212,212.483333,213,288.233333,296,267.300000,293,...,343.250000,345,237.30,241,238.716667,240,237.700000,239,162.96,7.82
2015-01-01 02:00:00,234.183333,249,192.966667,213,209.916667,213,293.133333,306,310.666667,371,...,343.950000,346,237.45,239,231.783333,236,238.633333,239,159.79,6.03


Сгенерируем из полученных данных дополнительные признаки

In [12]:
# Разница температур между соседними датчиками в камерах
for x in range(1, 6):
    for n in range(1, 3):
        new_feat = f'T_{x}_{n+1}-' + f'T_{x}_{n}'
        train_gr[new_feat] = train_gr[f'T_{x}_{n+1}_mean'] - train_gr[f'T_{x}_{n}_mean']

# Средняя температура по камерам
for x in range(1, 6):
    t_datas = [f'T_{x}_{n}_mean' for n in range(1,4)]
    train_gr[str(x)+'_cam_T_mean'] = train_gr[t_datas].mean(axis=1)

# Разница средних температур соседних камер
for x in range(1, 6):
    if x != 5:
        feat1 = str(x)+'_cam_T_mean'
        feat2 = str(x+1)+'_cam_T_mean'
        train_gr['T'+str(x+1)+'-T'+str(x)] = (train_gr[feat2]) - (train_gr[feat1])

In [13]:
train_gr.describe().round(2)

,T_1_1_mean,T_1_1_max,T_1_2_mean,T_1_2_max,T_1_3_mean,T_1_3_max,T_2_1_mean,T_2_1_max,T_2_2_mean,T_2_2_max,...,T_5_3-T_5_2,1_cam_T_mean,2_cam_T_mean,3_cam_T_mean,4_cam_T_mean,5_cam_T_mean,T2-T1,T3-T2,T4-T3,T5-T4
count,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00,...,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00,35064.00
mean,250.18,255.93,250.09,255.60,250.25,255.75,349.78,358.24,349.72,358.04,...,0.06,250.18,349.78,501.17,349.60,249.69,99.60,151.39,-151.57,-99.90
std,31.45,32.01,30.24,30.66,30.12,30.54,40.95,42.05,39.31,39.73,...,23.72,27.27,30.58,51.61,30.22,26.69,40.95,60.28,60.39,40.56
min,-189.40,-164.00,-112.83,-89.00,-98.37,-73.00,-647.23,-535.00,-912.50,-780.00,...,-415.62,94.68,-91.65,67.08,19.76,119.98,-348.86,-249.57,-820.72,-366.57
25%,228.78,234.00,228.75,234.00,229.07,234.00,328.27,335.00,328.02,335.00,...,-5.60,229.28,328.86,464.82,328.39,229.17,71.86,110.84,-192.32,-127.49
50%,250.07,256.00,250.12,255.00,250.13,256.00,349.92,357.00,349.52,357.00,...,0.07,249.99,349.80,501.79,349.58,249.55,99.54,151.18,-152.42,-100.40
75%,271.53,277.00,271.50,277.00,271.52,277.00,371.78,379.00,371.97,379.00,...,5.57,271.31,371.00,537.01,370.88,270.06,127.25,191.79,-110.60,-72.54
max,706.37,724.00,695.73,762.00,642.42,665.00,1269.05,1302.00,1118.10,1179.00,...,396.13,424.08,628.92,1145.57,652.41,453.89,378.98,747.47,274.41,235.65


Данные поступают каждую минуту, пока образец находится в обжиговой машине. Пробу берут раз в час на выходе сырья из машины. Таким образом, проба, взятая в момент времени 15:05:00, будет соответствовать образцу, который находился в обжиговой машине с 14:06:00 до 15:05:00. В агрегированных данных эти показания соответствуют моменту времени 14:00:00 (до агрегации был сдвиг на 6 минут назад).

In [14]:
new_train_target_index = [str(time_shift(time, hours=1, minutes=5)) for time in raw_target.index]
new_test_target_index = [str(time_shift(time, hours=1, minutes=5)) for time in raw_test.index]

target = pd.DataFrame(raw_target.values, index=new_train_target_index, columns=['target'])
test = pd.DataFrame(raw_test.values, index=new_test_target_index, columns=['target'])

target.head()

,target
2015-01-03 23:00:00,392
2015-01-04 00:00:00,384
2015-01-04 01:00:00,393
2015-01-04 02:00:00,399
2015-01-04 03:00:00,400


In [15]:
# Соединим датасет с признаками с целевой переменной, отобрав только те показания сырья, по которым известна целевая метка.
train_data = train_gr.merge(target, left_index=True, right_index=True)
test_data = train_gr.merge(test, left_index=True, right_index=True)

train_data.head(3)

,T_1_1_mean,T_1_1_max,T_1_2_mean,T_1_2_max,T_1_3_mean,T_1_3_max,T_2_1_mean,T_2_1_max,T_2_2_mean,T_2_2_max,...,1_cam_T_mean,2_cam_T_mean,3_cam_T_mean,4_cam_T_mean,5_cam_T_mean,T2-T1,T3-T2,T4-T3,T5-T4,target
2015-01-03 23:00:00,272.300000,277,337.483333,341,267.816667,273,327.516667,334,330.016667,335,...,292.533333,334.555556,532.972222,340.500000,236.994444,42.022222,198.416667,-192.472222,-103.505556,392
2015-01-04 00:00:00,277.650000,278,293.216667,326,273.500000,274,320.266667,322,331.766667,335,...,281.455556,335.977778,570.566667,352.150000,235.200000,54.522222,234.588889,-218.416667,-116.950000,384
2015-01-04 01:00:00,272.233333,277,227.983333,252,265.650000,271,323.266667,326,334.516667,336,...,255.288889,334.155556,518.750000,358.555556,239.133333,78.866667,184.594444,-160.194444,-119.422222,393


Отделим от данных hold-out датасет для итоговой проверки качества модели

In [16]:
X_train, X_hold_out, y_train, y_hold_out = train_test_split(train_data.iloc[:, :train_data.shape[1]-1], 
                                                            train_data.target, 
                                                            test_size=0.1, 
                                                            shuffle=True, 
                                                            random_state=0
                                                            )

print("Размеры данных: ", X_train.shape, X_hold_out.shape)

Размеры данных:  (26265, 51) (2919, 51)


## Модель 1

Параметры модели были подобраны поиском по сетке. Качество будем проверять методом кросс-валидации, разбивая данные на 5 фолдов.

In [17]:
%%time

SK = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

model1 = LGBMRegressor(boosting_type='gbdt',
                       colsample_bytree=0.8,
                       learning_rate=0.05,
                       n_estimators=2300,
                       num_leaves=14,
                       subsample=0.6,
                       reg_lambda=7.5,
                       metric='mae',
                       random_state=0,
                       )

score1 = cross_val_score(model1, X_train, y_train, cv=SK, scoring='neg_mean_absolute_error', n_jobs=-1)
print('Mean MAE on cross-validation =', -round(score1.mean(), 3))

Mean MAE on cross-validation = 6.728
Wall time: 28.8 s


Проверим качество модели с подобранными параметрами на hold-out датасете. В качестве дополнительной метрики посчитаем r2-score.

In [18]:
%%time
model1.fit(X_train, y_train)

hold_out_pred1 = model1.predict(X_hold_out)
mae_score1 = mean_absolute_error(hold_out_pred1, y_hold_out)
r2_score1 = r2_score(hold_out_pred1, y_hold_out)
print('MAE = ', round(mae_score1, 3), '\n', 'r2_score = ', round(r2_score1, 3), sep='')

MAE = 6.684
r2_score = 0.959
Wall time: 7.39 s


## Модель 2

In [19]:
model2 = CatBoostRegressor(n_estimators=2000,
                           learning_rate=0.1, 
                           loss_function='RMSE',
                           reg_lambda=5,
                           bootstrap_type='Bernoulli',
                           subsample=0.8,
                           random_state=0,
                           verbose=False
                          )

In [20]:
# %%time

# #выполняется долго
# score2 = cross_val_score(model2, X_train, y_train, cv=SK, scoring='neg_mean_absolute_error', n_jobs=-1)
# print('Mean MAE on cross-validation =', -round(score2.mean(), 3))

In [21]:
%%time
model2.fit(X_train, y_train)

hold_out_pred2 = model2.predict(X_hold_out)
mae_score2 = mean_absolute_error(hold_out_pred2, y_hold_out)
r2_score2 = r2_score(hold_out_pred2, y_hold_out)
print('MAE = ', round(mae_score2, 3), '\n', 'r2_score = ', round(r2_score2, 3), sep='')

MAE = 6.636
r2_score = 0.96
Wall time: 4min 41s


Оценка качества усредненных по двум моделям предсказаний на hold_out датасете.

In [22]:
pred_mean = (hold_out_pred1 + hold_out_pred2) / 2
mae_score_mean = mean_absolute_error(pred_mean, y_hold_out)
r2_sc_mean = r2_score(pred_mean, y_hold_out)
print('MAE = ', round(mae_score_mean, 3), ' r2 score = ', round(r2_sc_mean, 3))

MAE =  6.448  r2 score =  0.962


## Предсказания на тестовой выборке

В качестве предсказаний на тестовой выборке будем также брать средние предсказания двух моделей.

In [23]:
X_test = test_data.iloc[:, :test_data.shape[1]-1]

test_pred1 = model1.predict(X_test)
test_pred2 = model2.predict(X_test)

test_pred_mean = (test_pred1 + test_pred2) / 2

submit_data = pd.DataFrame(test_pred_mean, columns=['target'], index=raw_test.index)
submit_data.to_csv('submit_data.csv', header=None)
submit_data.head(3)

,target
timestamp,
2018-05-04 00:05:00,449.836178
2018-05-04 01:05:00,442.347370
2018-05-04 02:05:00,432.301965


## Как можно улучшить модель

Если иметь больше информации о производственном процессе, то можно более разумно подойти к генерации признаков. Например, температуру датчиков усреднять не по часу, а по тому интервалу времени, за который сырье прошло конкретную камеру. Для этого нужно знать сколько времени сырье находится в отдельно взятой камере.